<a href="https://colab.research.google.com/github/iyoushe1703/glassdoor-data-science-jobs-predictor/blob/main/Ken_Jee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
pd.set_option('display.max_columns', 3000)
pd.set_option('max_rows', 1000)

In [31]:
df = pd.read_csv("glassdoor_jobs.csv")

# Cleaning Salary Estimates Column

#### Removing missing values

In [32]:
# method 1
#df.drop(df.index[df['Salary Estimate'] == '-1'])

In [33]:
# method 2
df = df[df['Salary Estimate'] != '-1']

In [ ]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

#### Removing "Glassdoor Estimate"

In [35]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

#### Removing $ and K

In [36]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.replace('$', '').replace('K', ''))

#### Creating a column to indicate 'per hour' and 'Employer Provided Salary' entries in the ```Salary Estimate``` column

In [37]:
df['Per Hour'] = df['Salary Estimate'].apply(lambda x: 1 if 'Per Hour'.lower() in x.lower() else 0)

In [38]:
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary'.lower() in x.lower() else 0)

### Removing Employer Provided Salary from Salary Estimate

In [39]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.lower().replace("per hour", ""))

In [40]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.lower().replace("employer provided salary:", ""))

#### Creating ```min_salary```, ```max_salary``` and ```avg_salary``` columns

In [41]:
df['min_salary'] = df['Salary Estimate'].apply(lambda x: x.split('-')[0])

In [43]:
df['min_salary'] = df['min_salary'].astype(int)

In [44]:
df['max_salary'] = df['Salary Estimate'].apply(lambda x: x.split('-')[-1])

In [45]:
df['max_salary'] = df['max_salary'].astype(int)

In [46]:
df["avg_salary"] = (df['min_salary'] + df['max_salary'])/2

### Cleaning ```Company Name```

In [47]:
df['company_txt'] = df['Company Name'].apply(lambda x: x.split("\n")[0])

### Separating State from city names

In [48]:
df['job_state'] = df['Location'].apply(lambda x: x[-2:])

### Make a new column that indicates whether the job location is at the company headquarters

#### ```np.select``` is more efficient than ```.apply()```

In [49]:
import numpy as np

condition = [df['Location'] == df['Headquarters'],
             df['Location'] != df['Headquarters']]

outputs = [1, 0]

df['job_at_hq'] = np.select(condition, outputs)

### Company age


In [ ]:
df['Founded'].apply(lambda x: x if x < 1 else 2020 - x)

# Parsing Job Description

In [62]:
# checking if python is in the job description
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [71]:
# checking if R is in the job description
df['rstudio'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() else 0)

In [72]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [73]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [ ]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

# Export cleaned ```df``` to a .csv file

In [81]:
df.to_csv("glassdoor_cleaned.csv", index = False)